In [1]:
import pandas as pd
from elftools.elf.elffile import ELFFile

In [2]:
csv_name = "../faults_eddi_cfcss.csv"
bin_name = "../out_eddi_cfcss_1.elf"

data = pd.read_csv(csv_name)
data

,attempt,stop_addr,stop_fn,delay,target,bitflip,code,seed
0,0,0x08003146,prvIdleTask,0.153,0x2000b428,1,0,3735928559
1,1,0x0800053c,prvInsertBlockIntoFreeList,0.583,0x2000a608,128,0,3735928559
2,2,0x080030f0,prvIdleTask,0.381,0x20001d37,128,1,3735928559
3,3,0x0800d100,prvCheckTasksWaitingTermination_dup,0.653,0x20007e91,2,0,3735928559
4,4,memset,"<ucHeap+3040>,",0.057,0x20008a17,4,0,3735928559
...,...,...,...,...,...,...,...,...
46900,46900,0x08013d48,vQueueUnregisterQueue_dup,0.454,0x20000c4d,8,1,3735928559
46901,46901,0x08001efc,xTaskIncrementTick,0.345,0x200016f3,16,1,3735928559
46902,46902,0x080030f0,prvIdleTask,0.996,0x20000aec,4,1,3735928559
46903,46903,0x08002d7e,vTaskSwitchContext,0.433,0x20000588,4,1,3735928559


In [3]:
symtable = dict()

with open(bin_name, 'rb') as elffile:
    for section in ELFFile(elffile).iter_sections():
        try:
            for sym in section.iter_symbols():
                if (sym.name != "ucHeap" and sym.entry['st_info']['type'] == 'STT_OBJECT'):
                    symtable[sym.name] = {"addr": sym.entry['st_value'], "size": sym.entry['st_size'] * 8}
        except: 
            continue

In [4]:
data_size = data.shape[0]

new_data = [None for i in range(data_size)]

for idx, row in data.iterrows():
    for sym in symtable.keys():
        try:
            if int(row['target'][2:], base=16) in range(symtable[sym]['addr'], symtable[sym]['addr']+symtable[sym]['size'], 1):
                #print(symtable[sym])
                new_data[idx] = sym
                break
        except:
            break
    print(str(idx) + "/" + str(data_size), end = "\r")

data['struct'] = new_data

In [5]:
data.to_csv("faults_eddi_cfcss_names.csv")